In [2]:
from __future__ import print_function

import pandas as pd
import bqplot as bq
import numpy as np
from IPython.display import display
from rawPy.inversion import rsf_inversion
from functools import partial
from rawPy import rawPy as rp
import ipywidgets as widgets
from bqplot.interacts import BrushSelector,BrushIntervalSelector,PanZoom,IndexSelector


# Initialise inversion API
rsf = rsf_inversion()
# prepare files to save data
# parameter output


### 1. Import the data

In [3]:
path = widgets.Textarea(description='Enter exp path: ',placeholder='/path/to/file/',layout=widgets.Layout(width='500px', height='80px', border='red'))
exp_name = widgets.Text(description='exp name')

load = widgets.Button(description='load')
out_data = widgets.Output(layout={'border': '1px solid black'})
@out_data.capture()
def open_file(b):
    path_to_exp = (path.value+exp_name.value)
    data = rp.load_data(path_to_exp)
    load.out = data
load.on_click(open_file)
top = widgets.HBox([path,exp_name,load])
bottom = widgets.HBox([out_data])
widgets.VBox([top,bottom])

#widgets.HBox([out_data])

**Example:**    
**Enter exp path**: C:/users/marco/Dropbox/PythonModules/rawPy/Example/  
**exp name**: b754_data_rp

### 2. Select the rows of the velocity step  
- First select exp interval with plot on the left  
- The plot on right will show the selected interval  
- Use the tools to:  
    - Pan
    - Zoom
        - It will show below the rows selected
        - To come back to starting scale click outside the zoomed window
    - Select
        - shows a bar to pick a point
        - click on the point to see the row number and value of friction (the bar will freeze)
        - click on canvas to select another point
- When selecting another interval on the left plot, use the zoom tool with a single click on the right plot to update the scale

In [4]:
#%%

##
# Newer version using the bqplot figure object
# What works:
# (1) The communication between the two figures works well
# (2) The toggle buttons allow to change tool well 
# problem 1: when panning the scale remains the one fixed bu the panning tool 
# problem 2 : using the index gives the right index but the bar always comes back to the beginning of the plot
## 
import bqplot as bq
import numpy as np 
import ipywidgets as widgets
from bqplot.interacts import BrushSelector,BrushIntervalSelector,PanZoom,IndexSelector
from IPython.display import display

######################################################################
# set up the plot on the left (general overview)
######################################################################
### 
# USe button to activate the plots after the data have been loaded up
plot_button = widgets.Button(description='activate plot')
output = widgets.Output()
@output.capture()
def plot_raw(b):
    
    # define three arrays to use in the plot 
    # the general plot will be in displacement to better visualize
    # the second plot in row number to have a continous quantity to pick the intervals
    x = load.out.records_na.reshape(len(load.out.records_na))
    y = load.out.friction_na.reshape(len(load.out.friction_na))
    z = load.out.lp_disp_mm.reshape(len(load.out.ec_disp_mm))
    
    x_sc = bq.LinearScale()
    y_sc = bq.LinearScale()
    
    x_ax = bq.Axis(scale=x_sc, label='Displacement [mm]')
    y_ax = bq.Axis(scale=y_sc,orientation='vertical', label='Friction')
    
    # decimate the array to allow quick plotting
    pt = 50
    line = bq.Lines(x=z[::pt],y=y[::pt],scales={'x':x_sc,'y':y_sc},colors=['black'])
    
    fig = bq.Figure(marks=[line],axes=[x_ax,y_ax])
    
    intsel = BrushIntervalSelector(marks=[line], scale=x_sc,color='red')
    fig.interaction=intsel
    
    def update_bottom_plot(*arg):
        brushing = intsel.brushing
        
        if not brushing:
            if line.selected is not None:
                start_ix, end_ix = line.selected[0], line.selected[-1]
            else:
                start_ix, end_ix = 0, -1
            with line.hold_sync():
                # multiply the index for the decimation window to show all the data in the zoom plot
                line2.x = x[start_ix*pt:end_ix*pt]
                line2.y = y[start_ix*pt:end_ix*pt]
                
    # Print the intervals of the zoom window    
    tx_range_s = widgets.Text(description='Int start')
    tx_range_e = widgets.Text(description='Int end')
    ## Call back handler for the interactions
    def test_callback(change):
        tx_range_s.value = str(np.round(change.new[0],4))
        tx_range_e.value = str(np.round(change.new[1],4))
                
    intsel.observe(update_bottom_plot, 'brushing')
    intsel.observe(test_callback, names=['selected'])
    
    
    ######################################################################
    # set up the plot on the right (zoom plot)
    ######################################################################
    pippo = [] # pass an epty list at the beginnin to avoid double plotting
    x_sc2 = bq.LinearScale()
    y_sc2 = bq.LinearScale()
    
    x_ax2 = bq.Axis(scale=x_sc2,orientation='horizontal',label='Row number')
    y_ax2 = bq.Axis(scale=y_sc2,orientation='vertical', label='Friction')
    
    line2 = bq.Lines(x=pippo,y=y,scales={'x':x_sc2,'y':y_sc2},marker='circle',colors=['black'])
    fig2 = bq.Figure(marks=[line2],axes=[x_ax2,y_ax2])
    
    # define the interactions
    panzoom = PanZoom(scales={'x':[x_sc2],'y':[y_sc2]})
    index_sel = IndexSelector(scale=x_sc2,marks=[line2])
    brush_sel = BrushSelector(x_scale=x_sc2, y_scale=y_sc2, marks=[line2])
    
    #### index selector ###
    # create widget to show values
    tx_rowpick = widgets.Text(description='Picked row')
    tx_friction = widgets.Text(description='Friction')
    ## Call back handler for the interactions
    # In the double_zoom.ipynb there is an example on how to slice and get the row number from the displacement
    # here it is not possible since displacement is not continous 
    def test_callback(change):      
        tx_rowpick.value=str(int(change.new))
        tx_friction.value=str(np.round(line2.y[int(change.new)-int(line2.x[0])],5))
    index_sel.observe(test_callback, names=['selected'])  
            
    #### brush selector ###
    def update_plot(*arg):
        brushing = brush_sel.brushing
        if not brushing:
            if line2.selected is not None:
                minx = float(brush_sel.selected[0][0])
                maxx = float(brush_sel.selected[1][0])
                miny = float(brush_sel.selected[0][1])
                maxy = float(brush_sel.selected[1][1])
            else:
                minx = float(np.min(line2.x))
                maxx = float(np.max(line2.x))
                miny = float(np.min(line2.y))
                maxy = float(np.max(line2.y)) 
            with line2.hold_sync():
                line2.scales['x'].min = minx
                line2.scales['x'].max = maxx
                line2.scales['y'].min = miny
                line2.scales['y'].max = maxy

    # show the row number of the starting and ending of the interval selected
    tx_zint_s = widgets.Text(description='Row start')
    tx_zint_e = widgets.Text(description='Row end')
 
    def int_callback(change):
        tx_zint_s.value = str(int(change.new[0,0]))
        tx_zint_e.value = str(int(change.new[1,0]))
        
    brush_sel.observe(int_callback, names=['selected'])
    brush_sel.observe(update_plot, 'brushing')
    ############################
    # set up the toggle button to allow interacting with plot
    from collections import OrderedDict
    selection_interacts = widgets.ToggleButtons(options=OrderedDict([('none',None),('PanZoom',panzoom),
                                                                     ('Zoom',brush_sel),('Select',index_sel)]))
    
    
    widgets.link((selection_interacts, 'value'), (fig2, 'interaction'))
    
    # set up the widgets grid
    grid = widgets.GridspecLayout(4,6,width='auto',height='700px')

    grid[0,:2]=fig
    grid[1,0]=tx_range_s
    grid[2,0]=tx_range_e
    
    grid[0,2:]=fig2
    grid[1,2:]=selection_interacts
    grid[2,2]=tx_zint_s
    grid[3,2] =tx_zint_e
    grid[2,3]= tx_rowpick
    grid[3,3]=tx_friction
    
    
    display(grid)
    
plot_button.on_click(plot_raw)

widgets.VBox([plot_button,output])
#left = widgets.VBox([deb2,deb,fig2,selection_interacts])
#widgets.HBox([fig,left],align_self='stretch')




### 3. RSF tool

In [5]:

################################  set buttons #############################################

row_s = widgets.IntText(description='start row',layout=widgets.Layout(width='auto', height='auto'))
row_e = widgets.IntText(description='end row',layout=widgets.Layout(width='auto', height='auto'))
row_v = widgets.IntText(description='velocity row',layout=widgets.Layout(width='auto', height='auto'))
mu_0 = widgets.FloatText(description=r'$\mu_0$',layout=widgets.Layout(width='auto', height='auto'))

k = widgets.FloatText(description=r'k [$\mu /\mu m]$',layout=widgets.Layout(width='auto', height='auto'))
v_0 = widgets.IntText(description=r'$V_0 [\mu m/s]$',layout=widgets.Layout(width='auto', height='auto'))
v_1 = widgets.FloatText(description=r'$V_1 [\mu m/s]$',layout=widgets.Layout(width='auto', height='auto'))
evolution_law = widgets.Dropdown(options=['aging','slip'],description='Evolution law',value='aging')

solver_mode = widgets.Dropdown(options=['dense','step','bayes'],description='Solver mode',value='step')
a = widgets.FloatText(description='(a)',layout=widgets.Layout(width='auto', height='auto'))
b = widgets.FloatText(description='(b)',layout=widgets.Layout(width='auto', height='auto'))
dc = widgets.IntText(description='(dc)',layout=widgets.Layout(width='auto', height='auto'))

a_s = widgets.FloatSlider(description='(a)',min=0,max=0.1,step=0.0001,layout=widgets.Layout(width='auto', height='auto'))
b_s = widgets.FloatSlider(description='(b)',min=0,max=0.1,step=0.0001,layout=widgets.Layout(width='auto', height='auto'))
dc_s = widgets.IntSlider(description='(dc)',min=0,max=500,step=1,layout=widgets.Layout(width='auto', height='auto'))

link_a = widgets.jslink((a,'value'),(a_s,'value'))
link_b = widgets.jslink((b,'value'),(b_s,'value'))
link_dc = widgets.jslink((dc,'value'),(dc_s,'value'))


##################################  forward model ###########################################################

f_m = widgets.Button(description='Forward model',button_style='info',layout=widgets.Layout(width='auto', height='auto'))
output = widgets.Output()
@output.capture(clear_output=True)
def forward_model(f_m):
    ## interval data for modeling ##
    int_mu = load.out.friction_na[row_v.value:row_e.value]
    int_time = load.out.time_s[row_v.value:row_e.value]-load.out.time_s[row_v.value:row_e.value][0]
    int_mu = int_mu.reshape(len(int_mu))
    int_time = int_time.reshape(len(int_time))
    ## interval data for plotting ##
    int_mu_p = load.out.friction_na[row_s.value:row_e.value]
    int_time_p = load.out.time_s[row_s.value:row_e.value]-load.out.time_s[row_s.value:row_e.value][row_v.value-row_s.value]
    int_mu_p = int_mu_p.reshape(len(int_mu_p))
    int_time_p = int_time_p.reshape(len(int_time_p))
 
    t = np.linspace(0, int_time[-1], int(1e3))
    params = {
        "a" : a.value,
        "b" : b.value,
        "Dc": dc.value * 1e-6  , # [m]
        "k" : k.value / 1e-6, # [friction/m]??
        "mu0": mu_0.value,
        "V0": v_0.value * 1e-6,  #[m/s]
        "V1": v_1.value * 1e-6,  #[m/s]
        "eta": 0,
    }
    
    # Set model parameters
    rsf.set_params(params)

    # Select ageing law
    rsf.set_state_evolution(evolution_law.value)

    # Set initial values (V0, theta0), taken at steady-state
    y0 = [params["V0"], params["Dc"] / params["V0"]]
    rsf.set_initial_values(y0)

    # Perform forward model
    result = rsf.forward(t,mode="step")

    t = result['t']
    mu = result['mu']
    
    int_disp_p = (int_time_p*params['V1'])/1e-6   # [mic/s]*t
    disp = (t*params['V1'])/1e-6
    
    line.x=int_disp_p
    line.y=int_mu_p
    line2.x=disp
    line2.y=mu
    ys.min = np.min(int_mu)
    
    #print (params) # see how to print on the side of the figure
    out_params_f = pd.Series(params) # see if to use a dictionary or a series
    
    out_raw = {
        'int_mu': int_mu,
        'int_time' : int_time,
        'mu_model' : mu,
        't_model' : t
    }
    
    # stores all the current values in the event button 
    f_m.out_params_f = [params,out_raw]
    
f_m.on_click(forward_model)

########################################## inversion model ########################################################

inv_m = widgets.Button(description='Inversion',button_style='danger',layout=widgets.Layout(width='auto', height='auto'))
reset_inv = widgets.Button(description='Reset Inversion', button_style='danger',layout=widgets.Layout(width='auto', height='auto'))
def reset_inversion(b):
    line3.x=[]
    line3.y=[]
reset_inv.on_click(reset_inversion)

out_inv = widgets.Output(layout={'border': '1px solid black'})
@out_inv.capture(clear_output=True)
def simple_inversion(inv_m):
    f_m_params = f_m.out_params_f
    #t = np.linspace(0, int_time[-1], int(1e3))
    #t = t.reshape(len(t))
    t = np.linspace(0, f_m_params[1]['int_time'][-1], int(1e3))
    # Set model parameters
    params = f_m_params[0]
    rsf.set_params(params)
    
    # Select ageing law
    rsf.set_state_evolution(evolution_law.value)
    
    # Set initial values (V0, theta0), taken at steady-state
    y0 = [params["V0"], params["Dc"] / params["V0"]]
    rsf.set_initial_values(y0)
    
    # Perform forward model
    result = rsf.forward(t)
    
    # Construct our data dictionary
    data_dict = {"mu": f_m_params[1]['int_mu'], "t": f_m_params[1]['int_time']}
    
    # The parameters to invert for
    inversion_params = ("a", "b", "Dc", "k")
    
    # Perform the inversion. The results are given as a dictionary
    # in pairs of (value, uncertainty)
    
    if solver_mode.value == 'bayes':
        out,mu_model,t=rsf.inversion(data_dict, inversion_params,bayes=True,plot=False,mode='dense')
    else:
        out,mu_model,t=rsf.inversion(data_dict, inversion_params,plot=False,mode=solver_mode.value)
    
    disp_inv = (t*params['V1'])/1e-6 # trick to show disp based on shear velocity (not very accurate)
    
    line3.x = disp_inv
    line3.y = mu_model
    
    inv_m.out_params_inv = out
    
    print('Inversion results')
    print('')
    print('a: %0.5f '%out['a'][0])
    print('a STD: %s'%out['a'][1])
    print('-------------')
    print('b: %0.5f'%out['b'][0])
    print('b STD: %s'%out['b'][1])
    print('-------------')
    print('dc: %0.5f [micron]'%(float(out['Dc'][0])/1e-6))
    print('dc STD: %s'%out['Dc'][1])
    print('-------------')
    print('(a-b): %0.5f'%(out['a'][0]-out['b'][0]))
    
inv_m.on_click(simple_inversion)


########################################## add data to dictionary to be saved  ##################################################

add_step = widgets.Button(description='Add Step',button_style='success',layout=widgets.Layout(width='auto', height='auto'))

class Counter:
   def __init__(self, initial=0):
      self.value = initial

   def increment(self, amount=1):
      self.value += amount
      return self.value

   def __iter__(self, sentinal=False):
      return iter(self.increment, sentinal)

output3 = widgets.Output()
@output3.capture(clear_output=True)
def add_step_f(counter, w):
    counter.increment()  
    # there should be a better way to do this 
    to_save = {'a':inv_m.out_params_inv['a'][0],
               'aSTD':inv_m.out_params_inv['a'][1],
               'b':inv_m.out_params_inv['b'][0],
               'bSTD':inv_m.out_params_inv['b'][1],
               'Dc':inv_m.out_params_inv['Dc'][0],
               'DcSTD':inv_m.out_params_inv['Dc'][1],
               'k':inv_m.out_params_inv['k'][0],
               'kSTD':inv_m.out_params_inv['k'][1],
               'RowS':row_s.value,
               'RowE':row_e.value,
               'mu_0':mu_0.value,
               'V0': v_0.value,
               'V1':v_1.value
    }
    ser = pd.Series(to_save)
    df.iloc[counter.value]=ser 
    

names = ['a','aSTD','b','bSTD','Dc','DcSTD','k','kSTD','RowS','RowE','mu_0','V0','V1']
df = pd.DataFrame(columns=names,index=np.arange(100))
counter = Counter()
add_step.on_click(partial(add_step_f, counter))

########################################## save data to file #########################################################
# take the path from the loading experiment and save the file in the same directory
save = widgets.Button(description='Save data', button_style='primary',layout=widgets.Layout(width='auto', height='auto'))
output4 = widgets.Output()
@output4.capture()
def save_data(df,b):
    print(df)
    df = df.drop([0])
    path_to_save = (path.value+'rsf_data_table.csv')
    df.iloc[:counter.value].to_csv(path_to_save)

save.on_click(partial(save_data,df))

######################################### plot ###########################################
x = np.zeros(1)
y = np.zeros(1) #two random walks
x1 = np.zeros(1)
y1 = np.zeros(1)
x2 = np.zeros(1)
y2 = np.zeros(1)

xs = bq.LinearScale()
ys = bq.LinearScale()

line = bq.Lines(x=x, y=y, scales={'x': xs, 'y': ys}, colors=['black'],display_legend=True,labels=['Exp. data'])
line2 =bq.Lines(x=x1, y=y1, scales={'x': xs, 'y': ys}, colors=['green'],display_legend=True,labels=['Forward model']) 
line3 =bq.Lines(x=x2, y=y2, scales={'x': xs, 'y': ys}, colors=['red'],display_legend=True,labels=['Inversion']) 


xax = bq.Axis(scale=xs, label='Displacement [micron]')
yax = bq.Axis(scale=ys, orientation='vertical', tick_format='0.2f', label=r'Coefficient of friction')

fig = bq.Figure(marks=[line,line2,line3], axes=[xax, yax], animation_duration=1000)

#################################### display layout ################################################
grid = widgets.GridspecLayout(7,4,width='auto',height='700px')

grid[0,0]=row_s
grid[0,1]=row_e
grid[0,2]=row_v
grid[0,3]=mu_0

grid[1,0]=k
grid[1,1]=v_0
grid[1,2]=v_1
grid[1,3]=evolution_law

grid[2,0]=a
grid[2,1]=b
grid[2,2]=dc
grid[2,3]=solver_mode

grid[3,0]=a_s
grid[3,1]=b_s
grid[3,2]=dc_s

grid[4,0]=f_m
grid[4,1]=inv_m
grid[4,2]=add_step
grid[4,3]=save

grid[5,0:3]=fig
grid[5,-1]=out_inv
grid[6,-1]=reset_inv

display(grid,output)


GridspecLayout(children=(IntText(value=0, description='start row', layout=Layout(grid_area='widget001', height…

Output()